In [1]:
import numpy as np
from functools import cache
from itertools import zip_longest
import sys

%load_ext line_profiler

In [2]:
class Piece():
  rows: np.ndarray[np.uint8, np.uint8]
  columns: np.ndarray[np.uint8, np.uint8]
  num_rows: int
  num_columns: int

  def __init__(self, rows: tuple[tuple[bool]] | np.ndarray[np.uint8, np.uint8], columns: tuple[tuple[bool]] | np.ndarray[np.uint8, np.uint8]):
    rows = np.asarray(rows)
    columns = np.asarray(columns)

    # Trim piece to size
    while rows.size and ~np.any(rows[-1]) and columns.size and ~np.any(columns[-1]):
       rows = rows[:-1]
       columns = columns[:-1]

    while rows.size and ~np.any(rows[:,-1]) and columns.size and ~np.any(columns[:,-1]):
       rows = rows[:,:-1]
       columns = columns[:,:-1]
    
    # Edge case: when all rows or columns are trimmed from one side
    self.num_rows = 0 if not rows.size else rows.shape[0]
    self.num_columns = 0 if not columns.size else columns.shape[0]
    self.rows = np.packbits(rows, axis=0)
    self.columns = np.packbits(columns, axis=0)

  def __repr__(self):
    rows = np.unpackbits(self.rows, axis=0, count=self.num_rows)
    columns = np.unpackbits(self.columns, axis=0, count=self.num_columns)
    buffer = []
    for row, column in zip_longest(rows, columns):
        if row is not None:
          buffer.append(" " + " ".join('-' if elem else ' ' for elem in row))
        if column is not None:
          buffer.append(" ".join('|' if elem else ' ' for elem in column))
    return "\n".join(buffer)
  
  def __hash__(self):
     return hash((tuple(self.rows.flatten()), tuple(self.columns.flatten())))

  def __eq__(self, other):
      if isinstance(other, Piece):
          return np.array_equal(self.rows, other.rows) and np.array_equal(self.columns, other.columns)
      return NotImplemented
    
  def rotated(self):
    unpacked_rows = np.unpackbits(self.rows, axis=0, count=self.num_rows)
    unpacked_columns = np.unpackbits(self.columns, axis=0, count=self.num_columns)

    rows = np.rot90(unpacked_columns, 1)
    columns = np.rot90(unpacked_rows, 1)

    while rows.size and ~np.any(rows[0]) and columns.size and ~np.any(columns[0]):
      # top row is empty, so we can shift digit up
      rows = np.roll(rows, -1, 0)
      columns = np.roll(columns, -1, 0)

    return Piece(rows, columns)
  
  @cache
  def rotations(self):
    a = self
    b = a.rotated()
    c = b.rotated()
    d = c.rotated()
    return set((a, b, c, d))

pieces = [
  Piece(rows=((True, False), (True, False), (False, False)), columns=((True, True, False), (False, False, False))),
  Piece(rows=((False, False), (False, False), (False, False)), columns=((True, False, False), (True, False, False))),
  Piece(rows=((True, False), (True, False), (True, False)), columns=((False, True, False), (True, False, False))),
  Piece(rows=((True, False), (True, False), (True, False)), columns=((False, True, False), (False, True, False))),
  Piece(rows=((False, False), (True, False), (False, False)), columns=((True, True, False), (False, True, False))),
  Piece(rows=((True, False), (True, False), (True, False)), columns=((True, False, False), (False, True, False))),
  Piece(rows=((True, False), (True, False), (True, False)), columns=((True, False, False), (True, True, False))),
  Piece(rows=((True, False), (False, False), (False, False)), columns=((False, True, False), (False, True, False))),
  Piece(rows=((True, False), (True, False), (True, False)), columns=((True, True, False), (True, True, False))),
  Piece(rows=((True, False), (True, False), (True, False)), columns=((True, True, False), (False, True, False))),
]

print(sys.getsizeof(pieces[0].rows))

129


In [3]:
pieces[0]

 -
| |
 -

In [4]:
pieces[1]

|
|

In [5]:
pieces[9].rotations()

{   -
 | | |
  - -,
  -
 |  
  -
 | |
  -,
  -
 | |
  -
   |
  -,
  - -
 | | |
  -  }

In [6]:
pieces[0].rotations()

{ -
 | |
  -}

In [7]:
class Puzzle():
  NUM_ROWS = 5
  NUM_COLS = 4
  rows: np.ndarray[np.uint8, np.uint8]
  cols: np.ndarray[np.uint8, np.uint8]

  def __init__(
      self,
      rows=np.zeros((NUM_ROWS,5), dtype=np.uint8),
      cols=np.zeros((NUM_COLS,6), dtype=np.uint8),
    ):
    self.rows = np.packbits(rows, axis=0)
    self.cols = np.packbits(cols, axis=0)

  def __repr__(self):
    rows = np.unpackbits(self.rows, axis=0, count=self.NUM_ROWS)
    cols = np.unpackbits(self.cols, axis=0, count=self.NUM_COLS)
    buffer = []
    for row, column in zip_longest(rows, cols):
        if row is not None:
          buffer.append(" " + " ".join('-' if elem else '.' for elem in row))
        if column is not None:
          buffer.append(" ".join('|' if elem else '.' for elem in column))
    return "\n".join(buffer)

  def __hash__(self):
    return hash((tuple(self.rows.flatten()), tuple(self.cols.flatten())))

  def __eq__(self, other):
    if isinstance(other, Puzzle):
        return np.array_equal(self.rows, other.rows) and np.array_equal(self.cols, other.cols)
    return NotImplemented

  def with_piece(self, piece: Piece, toprow: int, topcol: int) -> "Puzzle":
    piece_rows = np.unpackbits(piece.rows, axis=0, count=piece.num_rows)
    piece_columns = np.unpackbits(piece.columns, axis=0, count=piece.num_columns)

    unpacked_rows = np.unpackbits(self.rows, axis=0, count=self.NUM_ROWS)
    unpacked_cols = np.unpackbits(self.cols, axis=0, count=self.NUM_COLS)

    if (toprow + piece_rows.shape[0] > unpacked_rows.shape[0]):
      return None
    if (topcol + piece_rows.shape[1] > unpacked_rows.shape[1]):
      return None
    if (toprow + piece_columns.shape[0] > unpacked_cols.shape[0]):
      return None
    if (topcol + piece_columns.shape[1] > unpacked_cols.shape[1]):
      return None
    
    puzzle_rows = unpacked_rows[toprow:toprow+piece_rows.shape[0], topcol:topcol+piece_rows.shape[1]]
    if np.any(np.logical_and(puzzle_rows, piece_rows)):
      return None

    puzzle_cols = unpacked_cols[toprow:toprow+piece_columns.shape[0], topcol:topcol+piece_columns.shape[1]]
    if np.any(np.logical_and(puzzle_cols, piece_columns)):
      return None
    
    np.logical_or(puzzle_rows, piece_rows, out=puzzle_rows)
    np.logical_or(puzzle_cols, piece_columns, out=puzzle_cols)

    return Puzzle(unpacked_rows, unpacked_cols)
  
  def spaces_left(self):
    unpacked_rows = np.unpackbits(self.rows, axis=0, count=self.NUM_ROWS)
    unpacked_cols = np.unpackbits(self.cols, axis=0, count=self.NUM_COLS)
    return np.size(unpacked_cols) - np.count_nonzero(unpacked_cols) + np.size(unpacked_rows) - np.count_nonzero(unpacked_rows)

In [8]:
%timeit Puzzle().with_piece(pieces[7], 2, 3)

85.5 µs ± 4.95 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [9]:
%lprun -f Puzzle.with_piece Puzzle().with_piece(pieces[7], 2, 3)

Timer unit: 1e-07 s

Total time: 0.0003013 s
File: C:\Users\Vidminas\AppData\Local\Temp\ipykernel_16496\3266338350.py
Function: with_piece at line 34

Line #      Hits         Time  Per Hit   % Time  Line Contents
    34                                             def with_piece(self, piece: Piece, toprow: int, topcol: int) -> "Puzzle":
    35         1        146.0    146.0      4.8      piece_rows = np.unpackbits(piece.rows, axis=0, count=piece.num_rows)
    36         1         84.0     84.0      2.8      piece_columns = np.unpackbits(piece.columns, axis=0, count=piece.num_columns)
    37                                           
    38         1         74.0     74.0      2.5      unpacked_rows = np.unpackbits(self.rows, axis=0, count=self.NUM_ROWS)
    39         1         69.0     69.0      2.3      unpacked_cols = np.unpackbits(self.cols, axis=0, count=self.NUM_COLS)
    40                                           
    41         1         52.0     52.0      1.7      if (topro

In [10]:
Puzzle()

 . . . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .

In [11]:
Puzzle().spaces_left()

49

In [12]:
Puzzle().with_piece(pieces[0], 1, 4)

 . . . . .
. . . . . .
 . . . . -
. . . . | |
 . . . . -
. . . . . .
 . . . . .
. . . . . .
 . . . . .

In [13]:
Puzzle().with_piece(pieces[7].rotated(), 0, 0)

 - - . . .
| . . . . .
 . . . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .

In [14]:
Puzzle().with_piece(pieces[3], 0, 0).with_piece(pieces[4].rotated(), 0, 1)

 - - - . .
. | | . . .
 - - . . .
. | . . . .
 - . . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .

In [15]:
Puzzle().with_piece(pieces[9].rotated().rotated().rotated(), 1, 0)

 . . . . .
. . . . . .
 . - . . .
| | | . . .
 - - . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .

In [42]:
from copy import copy

bad_puzzles = set()

# pick a puzzle piece
# choose rotation
# find top-left free corner
# put piece if it fits
# check for solved
def attempt(puzzle: Puzzle, used_pieces: dict[int, tuple]):
  if puzzle in bad_puzzles:
    return None
  if puzzle.spaces_left() == 2:
    return puzzle, used_pieces
  
  candidates = []
  for i in range(len(pieces)):
    if i in used_pieces:
      continue

    for rotation in pieces[i].rotations():
      for row in range(0, puzzle.NUM_ROWS):
        for col in range(0, puzzle.NUM_COLS):
          candidate = puzzle.with_piece(rotation, row, col)
    
          if candidate is not None:
            if candidate in bad_puzzles:
              continue

            candidate_used_pieces = copy(used_pieces)
            candidate_used_pieces[i] = (rotation, row, col)
            candidates.append((candidate, candidate_used_pieces))
    
  for candidate, candidate_used_pieces in candidates:
    result = attempt(candidate, candidate_used_pieces)
    if result is not None:
      return result

  bad_puzzles.add(puzzle)
  return None

In [44]:
rot9 = pieces[9].rotated().rotated().rotated()
puzzle = Puzzle().with_piece(rot9, 1, 0)
# solution = attempt(puzzle, {9: (rot9, 1, 0)})
# solution
%lprun -f attempt attempt(puzzle, {9: (rot9, 1, 0)})

*** KeyboardInterrupt exception caught in code being profiled.

Timer unit: 1e-07 s

Total time: 91.1 s
File: C:\Users\Vidminas\AppData\Local\Temp\ipykernel_16496\2297425621.py
Function: attempt at line 10

Line #      Hits         Time  Per Hit   % Time  Line Contents
    10                                           def attempt(puzzle: Puzzle, used_pieces: dict[int, tuple]):
    11      4286    1796496.0    419.2      0.2    if puzzle in bad_puzzles:
    12                                               return None
    13      4286    3254468.0    759.3      0.4    if puzzle.spaces_left() == 2:
    14                                               return puzzle, used_pieces
    15                                             
    16      4286      56640.0     13.2      0.0    candidates = []
    17     47141     786607.0     16.7      0.1    for i in range(len(pieces)):
    18     42856     697597.0     16.3      0.1      if i in used_pieces:
    19     28878     223008.0      7.7      0.0        continue
    20                                       

In [45]:
len(bad_puzzles)

4279

In [46]:
next(iter(bad_puzzles))

 - . - - .
| | | | . .
 - - . . .
| | | | . .
 - - - - .
. . | | | |
 . - - . -
. . | | | .
 . . - - .